In [1]:
import os
import numpy as np
import sys
print(sys.executable)
import tensorflow as tf
from tensorflow.keras import layers, models
import import_ipynb
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path


/bin/python3.11


2025-03-14 12:03:36.900934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741950216.943701  478353 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741950216.956506  478353 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 12:03:37.038182: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# print number of instances missing preterm label

# Load the preprocessed dataset
file_path = os.path.join(processed_data_path, 'target_preprocessed.npy')  # Update this with the actual path
target_data = np.load(file_path, allow_pickle=True)

# Count total instances
total_instances = len(target_data)

# Count instances where 'preterm' is None
none_preterm_count = sum(1 for entry in target_data if entry['preterm'] is None)

# Print results
print(f"Total instances: {total_instances}")
print(f"Instances with 'preterm' as None: {none_preterm_count}")


Total instances: 143
Instances with 'preterm' as None: 0


In [3]:

class Classifier:
    def __init__(self, encoder, input_shape, num_classes):
        self.encoder = encoder
        self.model = self._build_classifier(input_shape, num_classes)
    
    def _build_classifier(self, input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)

        # Freeze encoder weights
        self.encoder.trainable = False

        x = self.encoder(inputs, training=False)  # Use frozen encoder
        x = layers.Dense(32, activation="relu")(x)
        x = layers.Dense(16, activation="relu")(x)
        outputs = layers.Dense(num_classes, activation="softmax")(x)

        model = models.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
        return model


    def train(self, x_train, y_train, batch_size=64, epochs=50, validation_split=0.1):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)
        return history
    
    def evaluate(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)
    
    def predict(self, x):
        return self.model.predict(x)

# # Load the trained encoder from MaskedAutoencoder
# autoencoder = MaskedAutoencoder(input_dim=500)  # Use the correct input dimension

# # Define an input layer matching the expected shape
# input_layer = layers.Input(shape=(500,))  # Make sure this matches your signal window size

# # Pass the input through the autoencoder's encoder
# encoded_output = autoencoder.encoder(input_layer)

# # Create a new model for the encoder
# encoder = models.Model(input_layer, encoded_output)

encoder_path = "../models/encoder_model.keras"

if os.path.exists(encoder_path):
    # Load saved encoder
    print("Loading saved encoder...")
    encoder = tf.keras.models.load_model(encoder_path)
else:
    # Train autoencoder and extract encoder
    autoencoder = MaskedAutoencoder(input_dim=500)
    input_layer = layers.Input(shape=(500,))
    encoded_output = autoencoder.encoder(input_layer)
    encoder = models.Model(input_layer, encoded_output)

    # Save encoder
    print("Saving trained encoder...")
    encoder.save(encoder_path)

# Load the target dataset
save_dir = os.path.join(univariate_data_path, "target_univariate.npy")
target_data = np.load(save_dir, allow_pickle=True)

def create_windows(data, window_size=500, step=250):
    all_windows, labels = [], []
    for sample in data:
        signal, label = sample["signal"], sample["preterm"]
        
        # Ensure the label is a valid integer
        if label is None:  # Skip instances with None labels (just in case)
            continue
        
        for start in range(0, len(signal) - window_size + 1, step):
            window = signal[start : start + window_size]
            all_windows.append(window)
            labels.append(label)

    return np.array(all_windows, dtype=np.float32), np.array(labels, dtype=np.int32)

x_train, y_train = create_windows(target_data)  # Make sure this function is called!


# print(type(y_train[0]))
# # Initialize and train the classifier
num_classes = len(set(y_train))  # Get number of unique classes
# classifier = Classifier(encoder, input_shape=(500,), num_classes=num_classes)
# print(f"x_train type: {type(x_train)}, dtype: {x_train.dtype}, shape: {x_train.shape}")
# print(f"y_train type: {type(y_train)}, dtype: {y_train.dtype}, shape: {y_train.shape}")

# classifier.train(x_train, y_train)


Loading saved encoder...


2025-03-14 12:03:47.053195: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
from skopt import gp_minimize
from skopt.space import Real, Categorical
from skopt.utils import use_named_args
import numpy as np
import os

# Define search space: learning rate (log-uniform) & batch size (powers of 2)
search_space = [
    Real(1e-5, 1e-2, "log-uniform", name="learning_rate"),  # Learning rate
    Categorical([16, 32, 64, 128], name="batch_size")  # Only powers of 2
]

# Objective function for optimization
@use_named_args(search_space)
def objective(learning_rate, batch_size):
    print(f"Testing: Learning Rate={learning_rate:.6f}, Batch Size={batch_size}")

    # Create classifier with frozen encoder
    classifier = Classifier(encoder, input_shape=(500,), num_classes=num_classes)

    # Compile with the given learning rate
    classifier.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Train the model
    history = classifier.train(x_train, y_train, batch_size=batch_size, epochs=20, validation_split=0.1)

    # Return final validation loss (minimization objective)
    final_val_loss = history.history["val_loss"][-1]
    print(f"Validation Loss: {final_val_loss:.6f}\n")

    return final_val_loss

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=30, random_state=42)

# Print best hyperparameters
print("\nBest Hyperparameters:")
print(f"Learning Rate: {result.x[0]:.6f}")
print(f"Batch Size: {result.x[1]}")

# Plot optimization results
import matplotlib.pyplot as plt

plt.plot(result.func_vals)
plt.xlabel("Iteration")
plt.ylabel("Validation Loss")
plt.title("Bayesian Optimization Progress")
plt.show()


Testing: Learning Rate=0.002453, Batch Size=16
Epoch 1/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 37s 1ms/step - accuracy: 0.7383 - loss: 0.5409 - val_accuracy: 0.7075 - val_loss: 0.6421
Epoch 2/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.7462 - loss: 0.5193 - val_accuracy: 0.7133 - val_loss: 0.6553
Epoch 3/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.7459 - loss: 0.5179 - val_accuracy: 0.7098 - val_loss: 0.6458
Epoch 4/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - accuracy: 0.7465 - loss: 0.5182 - val_accuracy: 0.7134 - val_loss: 0.6389
Epoch 5/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.7455 - loss: 0.5185 - val_accuracy: 0.7129 - val_loss: 0.6532
Epoch 6/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step - accuracy: 0.7458 - loss: 0.5176 - val_accuracy: 0.7130 - val_loss: 0.6504
Epoch 7/20
34247/34247 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - accuracy: 0.7460 - loss: 0.5166 - val_accuracy: 0.7125 - val_loss: 0.6410
Epoch 8/20
34247/34247

KeyboardInterrupt: 

In [11]:
# Define save path
classifier_save_path = "../models/classifier_model.keras"

# Save the trained classifier
classifier.model.save(classifier_save_path)

print(f"Classifier model saved at: {classifier_save_path}")

Classifier model saved at: ../models/classifier_model.keras
